# Data Processing Notebook

This notebook performs data transformation, dimensional modeling, and analytics processing.

## Steps:
1. Load validated raw data
2. Create dimensional model (facts and dimensions)
3. Customer segmentation analysis
4. Product performance analytics
5. Generate business metrics
6. Store processed data for analytics


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import logging
from datetime import datetime
import json

# Build the Spark Session using AWS Keys from Databricks Secrets
spark = SparkSession.builder \
    .appName("PsychoBunny-DataIngestion") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key", dbutils.secrets.get(scope="aws-keys", key="aws-access-key")) \
    .config("spark.hadoop.fs.s3a.secret.key", dbutils.secrets.get(scope="aws-keys", key="aws-secret-key")) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Spark session initialized with Delta Lake and AWS S3 support")


INFO:__main__:Spark session initialized with Delta Lake and AWS S3 support
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


In [0]:
# Configuration
S3_BUCKET = "psycho-bunny-data-lake"
RAW_DATA_PATH = "s3://psycho-bunny-data-lake/raw-data/"
PROCESSED_DATA_PATH = f"s3://{S3_BUCKET}/processed-data/"
config_path = "/dbfs/FileStore/tables/pipeline_config.json"

# Load configuration
with open(config_path, 'r') as f:
    config = json.load(f)

business_rules = config['business_rules']


INFO:py4j.clientserver:Received command c on object id p0


In [0]:
access = dbutils.secrets.get("aws-keys", key="aws-access-key")
secret = dbutils.secrets.get("aws-keys", key="aws-secret-key")

spark.conf.set("fs.s3a.access.key", access)
spark.conf.set("fs.s3a.secret.key", secret)
spark.conf.set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")    


In [0]:
dbutils.fs.ls("s3://psycho-bunny-data-lake/raw-data/")

INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Received command c on object id p0


[FileInfo(path='s3://psycho-bunny-data-lake/raw-data/calendar/', name='calendar/', size=0, modificationTime=1750378984376),
 FileInfo(path='s3://psycho-bunny-data-lake/raw-data/customers/', name='customers/', size=0, modificationTime=1750378984376),
 FileInfo(path='s3://psycho-bunny-data-lake/raw-data/transactions/', name='transactions/', size=0, modificationTime=1750378984376)]

In [0]:
# Load raw data from Delta Lake
try:
    customers_df = spark.read.format("delta").load(f"{RAW_DATA_PATH}customers")
    transactions_df = spark.read.format("delta").load(f"{RAW_DATA_PATH}transactions")
    calendar_df = spark.read.format("delta").load(f"{RAW_DATA_PATH}calendar")
    
    logger.info("Raw data loaded successfully")
    logger.info(f"Customers: {customers_df.count()}, Transactions: {transactions_df.count()}, Calendar: {calendar_df.count()}")
except Exception as e:
    logger.error(f"Error loading raw data: {str(e)}")
    raise


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Raw data loaded successfully
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Customers: 2000, Transactions: 2823, Calendar: 6944
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


In [0]:
#Test Check
display(transactions_df.limit(50))

INFO:py4j.clientserver:Received command c on object id p0


ORDERNUMBER,QUANTITYORDERED,ORDERLINENUMBER,TOTAL_AMOUNT,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,source_file,ingestion_date,partition_date
10107,30,2,-1993,2/24/2003 0:00,1,2,2021,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,null,NYC,NY,10022,USA,NA,Yu,Kwai,Small,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10121,34,5,-92,5/7/2003 0:00,2,5,2021,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,null,Reims,null,51100,France,EMEA,Henriot,Paul,Small,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10134,41,2,-574,7/1/2003 0:00,3,7,2024,S10_1678,Lyon Souveniers,#ERROR!,27 rue du Colonel Pierre Avia,null,Paris,null,75508,France,EMEA,Da Cunha,Daniel,Medium,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10145,45,6,-623,8/25/2003 0:00,3,8,2020,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,null,Pasadena,CA,90003,USA,NA,Young,Julie,Medium,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10159,49,14,-442,10/10/2003 0:00,4,10,2022,S10_1678,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,null,San Francisco,CA,null,USA,NA,Brown,Julie,Medium,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10168,36,1,1540,10/28/2003 0:00,4,10,2022,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,null,Burlingame,CA,94217,USA,NA,Hirano,Juri,Medium,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10180,29,9,-1669,11/11/2003 0:00,4,11,2023,S10_1678,Daedalus Designs Imports,20.16.1555,"184, chausse de Tournai",null,Lille,null,59000,France,EMEA,Rance,Martine,Small,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10188,48,1,803,11/18/2003 0:00,4,11,2020,S10_1678,Herkku Gifts,#ERROR!,"Drammen 121, PR 744 Sentrum",null,Bergen,null,N 5804,Norway,EMEA,Oeztan,Veysel,Medium,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10201,22,2,1341,12/1/2003 0:00,4,12,2022,S10_1678,Mini Wheels Co.,6505555787,5557 North Pendale Street,null,San Francisco,CA,null,USA,NA,Murphy,Julie,Small,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01
10211,41,14,-741,1/15/2004 0:00,1,1,2023,S10_1678,Auto Canal Petit,(1) 47.55.6555,"25, rue Lauriston",null,Paris,null,75016,France,EMEA,Perrier,Dominique,Medium,de_shop_transactions_20230821.csv,2025-07-01T21:37:02.020173Z,2025-07-01


In [0]:
# Create customer dimension table
dim_customer = customers_df.select(
    col("customer_id").alias("customer_key"),
    col("first_name"),
    col("last_name"),
    concat_ws(" ", col("first_name"), col("last_name")).alias("full_name"),
    col("email"),
    col("company_name"),
    col("address"),  
    col("city"),
    col("state"),
    col("postal_code"),  
    col("phone1").alias("phone"),  
    col("ingestion_date").alias("created_date"),
    current_timestamp().alias("updated_date")
).dropDuplicates(["customer_key"])

logger.info(f"Created customer dimension with {dim_customer.count()} records")
display(dim_customer.limit(30))


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Created customer dimension with 2000 records
INFO:py4j.clientserver:Received command c on object id p0


customer_key,first_name,last_name,full_name,email,company_name,address,city,state,postal_code,phone,created_date,updated_date
aaron_kloska_aaron_kloska@kloska.net.au,Aaron,Kloska,Aaron Kloska,aaron_kloska@kloska.net.au,"Radecker, H Philip Jr",423 S Navajo St #56,Brookhill,QLD,4816,07-9896-4827,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
abel_maclead_amaclead@gmail.com,Abel,Maclead,Abel Maclead,amaclead@gmail.com,Rangoni Of Florence,37275 St Rt 17m M,Middle Island,NY,11953,631-335-3414,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
abraham_cratch_acratch@gmail.com,Abraham,Cratch,Abraham Cratch,acratch@gmail.com,"Cavuto, John A",41 Benedict St,Aldborough Ward,null,IG2 7QG,01599-245408,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adaline_galagher_adaline.galagher@galagher.com.au,Adaline,Galagher,Adaline Galagher,adaline.galagher@galagher.com.au,Debbie Reynolds Hotel,32716 N Michigan Ave #82,Barooga,NSW,3644,02-3225-1954,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adela_echegoyen_adela.echegoyen@echegoyen.org,Adela,Echegoyen,Adela Echegoyen,adela.echegoyen@echegoyen.org,Fpa Corp,128 W Kellogg Dr,Burnaby,BC,V5B 4L5,604-571-8392,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adelaide_ender_aender@gmail.com,Adelaide,Ender,Adelaide Ender,aender@gmail.com,Williams Design Group,175 N Central Ave,Greenslopes,QLD,4120,07-7538-5504,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adelina_nabours_adelina_nabours@gmail.com,Adelina,Nabours,Adelina Nabours,adelina_nabours@gmail.com,Courtyard By Marriott,80 Pittsford Victor Rd #9,Cleveland,OH,44103,216-230-4892,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adell_lipkin_adell.lipkin@lipkin.com,Adell,Lipkin,Adell Lipkin,adell.lipkin@lipkin.com,Systems Graph Inc Ab Dick Dlr,65 Mountain View Dr,Whippany,NJ,07981,973-654-1561,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adelle_nitcher_anitcher@aol.com,Adelle,Nitcher,Adelle Nitcher,anitcher@aol.com,John J Mccarthy Agency Inc,10 Midway Dr,Thorold,ON,L2V 5C7,905-998-3758,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z
adelle_schantini_adelle_schantini@yahoo.com,Adelle,Schantini,Adelle Schantini,adelle_schantini@yahoo.com,Creative Photography Inc,162 Grayson St,Greenlands Ward,null,FY2 0TD,01624-595660,2025-07-01T21:36:59.253098Z,2025-07-01T22:33:06.022711Z


In [0]:
# extract product family from product code
def extract_product_family(product_code):
    if product_code and "_" in product_code:
        return product_code.split("_")[0]
    return "UNKNOWN"

extract_product_family_udf = udf(extract_product_family, StringType())

# Create simple product dimension table
dim_product = transactions_df.select("PRODUCTCODE").distinct() \
    .withColumn("product_key", col("PRODUCTCODE")) \
    .withColumn("product_family", extract_product_family_udf(col("PRODUCTCODE"))) \
    .withColumn("product_line", lit("General")) \
    .withColumn("created_date", current_timestamp())

logger.info(f"Created product dimension with {dim_product.count()} records")
display(dim_product.limit(30))


INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Created product dimension with 109 records
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


PRODUCTCODE,product_key,product_family,product_line,created_date
S18_4600,S18_4600,S18,General,2025-07-01T22:33:18.787661Z
S18_1749,S18_1749,S18,General,2025-07-01T22:33:18.787661Z
S12_3891,S12_3891,S12,General,2025-07-01T22:33:18.787661Z
S18_2248,S18_2248,S18,General,2025-07-01T22:33:18.787661Z
S700_1138,S700_1138,S700,General,2025-07-01T22:33:18.787661Z
S32_1268,S32_1268,S32,General,2025-07-01T22:33:18.787661Z
S12_1099,S12_1099,S12,General,2025-07-01T22:33:18.787661Z
S18_2795,S18_2795,S18,General,2025-07-01T22:33:18.787661Z
S24_1937,S24_1937,S24,General,2025-07-01T22:33:18.787661Z
S32_3522,S32_3522,S32,General,2025-07-01T22:33:18.787661Z


In [0]:
from pyspark.sql.types import DoubleType

BUSINESS_RULES = {
    "high_value_customer_threshold": 10000,  # Customers who spent more than $10K
    "refund_restocking_fee_rate": 0.10,      # 10% restocking fee on refunds
    "large_order_threshold": 5000            # Orders above $5K are considered large
}

# Create fact table with business logic using actual DataFrame columns
fact_transactions = transactions_df.select(
    col("ORDERNUMBER").cast("long").alias("order_number"),
    concat_ws(" ", col("CONTACTFIRSTNAME"), col("CONTACTLASTNAME")).alias("customer_name"),
    col("ADDRESSLINE1").alias("address"),
    col("PHONE").alias("phone_number"),
    col("PRODUCTCODE").alias("product_code"),
    col("QUANTITYORDERED").alias("quantity"),
    # Calculate unit price from total amount and quantity
    (abs(col("TOTAL_AMOUNT")) / col("QUANTITYORDERED")).alias("unit_price"),
    col("TOTAL_AMOUNT").cast("double").alias("total_amount"),  # Use absolute value since amounts are negative
    col("DEALSIZE").alias("deal_size"),
    col("TERRITORY").alias("territory"),
    to_date(col("ORDERDATE"), "M/d/yyyy H:mm").alias("order_date"),  # Convert date format
    # Set default status and product line
    lit("COMPLETED").alias("status"),
    lit("General").alias("product_line"),
    current_timestamp().alias("created_date")
).withColumn(
    # Business rule: Classify transaction type based on negative amounts (refunds)
    "transaction_type",
    when(col("TOTAL_AMOUNT") < 0, "REFUND").otherwise("SALE")
).withColumn(
    # Business rule: Calculate restocking fee for refunds (10% of amount)
    "restocking_fee", 
    when(col("transaction_type") == "REFUND", 
         abs(col("TOTAL_AMOUNT")) * BUSINESS_RULES["refund_restocking_fee_rate"])
    .otherwise(0.0)
).withColumn(
    # Business rule: Calculate net amount after fees
    "net_amount",
    when(col("transaction_type") == "REFUND", 
         abs(col("TOTAL_AMOUNT")) - col("restocking_fee"))
    .otherwise(abs(col("TOTAL_AMOUNT")))
).withColumn(
    # Business rule: Flag large orders (>$5K)
    "is_large_order",
    when(abs(col("TOTAL_AMOUNT")) >= BUSINESS_RULES["large_order_threshold"], 'true')
    .otherwise('false')
)

logger.info(f"Created fact transactions with {fact_transactions.count()} records")
display(fact_transactions.limit(30))




INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Created fact transactions with 2823 records
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


order_number,customer_name,address,phone_number,product_code,quantity,unit_price,total_amount,deal_size,territory,order_date,status,product_line,created_date,transaction_type,restocking_fee,net_amount,is_large_order
10107,Kwai Yu,897 Long Airport Avenue,2125557818,S10_1678,30,66.43333333333334,-1993.0,Small,NA,2003-02-24,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,199.3,1793.7,false
10121,Paul Henriot,59 rue de l'Abbaye,26.47.1555,S10_1678,34,2.7058823529411766,-92.0,Small,EMEA,2003-05-07,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,9.200000000000001,82.8,false
10134,Daniel Da Cunha,27 rue du Colonel Pierre Avia,#ERROR!,S10_1678,41,14.0,-574.0,Medium,EMEA,2003-07-01,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,57.400000000000006,516.6,false
10145,Julie Young,78934 Hillside Dr.,6265557265,S10_1678,45,13.844444444444445,-623.0,Medium,NA,2003-08-25,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,62.300000000000004,560.7,false
10159,Julie Brown,7734 Strong St.,6505551386,S10_1678,49,9.020408163265307,-442.0,Medium,NA,2003-10-10,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,44.2,397.8,false
10168,Juri Hirano,9408 Furth Circle,6505556809,S10_1678,36,42.77777777777778,1540.0,Medium,NA,2003-10-28,COMPLETED,General,2025-07-02T02:46:39.362186Z,SALE,0.0,1540.0,false
10180,Martine Rance,"184, chausse de Tournai",20.16.1555,S10_1678,29,57.55172413793103,-1669.0,Small,EMEA,2003-11-11,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,166.9,1502.1,false
10188,Veysel Oeztan,"Drammen 121, PR 744 Sentrum",#ERROR!,S10_1678,48,16.729166666666668,803.0,Medium,EMEA,2003-11-18,COMPLETED,General,2025-07-02T02:46:39.362186Z,SALE,0.0,803.0,false
10201,Julie Murphy,5557 North Pendale Street,6505555787,S10_1678,22,60.95454545454545,1341.0,Small,NA,2003-12-01,COMPLETED,General,2025-07-02T02:46:39.362186Z,SALE,0.0,1341.0,false
10211,Dominique Perrier,"25, rue Lauriston",(1) 47.55.6555,S10_1678,41,18.073170731707318,-741.0,Medium,EMEA,2004-01-15,COMPLETED,General,2025-07-02T02:46:39.362186Z,REFUND,74.10000000000001,666.9,false


In [0]:
# Testing one example
from pyspark.sql import functions as F

#df=fact_transactions.filter(F.col('transaction_type') == 'REFUND')
df=spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}fact_transactions")
#display(df)
fact_transactions.printSchema()
df.printSchema()

INFO:py4j.clientserver:Received command c on object id p1
INFO:py4j.clientserver:Received command c on object id p0


root
 |-- order_number: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- product_code: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- total_amount: integer (nullable = true)
 |-- deal_size: string (nullable = true)
 |-- territory: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- status: string (nullable = false)
 |-- product_line: string (nullable = false)
 |-- created_date: timestamp (nullable = false)
 |-- transaction_type: string (nullable = false)
 |-- restocking_fee: double (nullable = true)
 |-- net_amount: double (nullable = true)
 |-- is_large_order: string (nullable = false)

root
 |-- product_code: string (nullable = true)
 |-- territory: string (nullable = true)
 |-- customer_key: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- order_number: long (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- status: string (nullable

In [0]:
# Customer segmentation based on spending
customer_segments = fact_transactions.filter(col("transaction_type") == "SALE") \
    .groupBy("customer_name") \
    .agg(
        sum("net_amount").alias("total_spent"),
        count("order_number").alias("total_orders"),
        avg("net_amount").alias("avg_order_value"),
        max("order_date").alias("last_order_date")
    ).withColumn(
        # Business rule: Customer value segmentation
        "customer_segment",
        when(col("total_spent") >= BUSINESS_RULES["high_value_customer_threshold"], "High Value")
        .otherwise("Regular")
    ).withColumn("created_date", current_timestamp())

logger.info(f"Created customer segments with {customer_segments.count()} customers")
display(customer_segments)

INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Created customer segments with 91 customers
INFO:py4j.clientserver:Received command c on object id p0


customer_name,total_spent,total_orders,avg_order_value,last_order_date,customer_segment,created_date
Elizabeth Devon,17910.0,14,1279.2857142857142,2005-04-08,High Value,2025-07-01T22:50:16.00517Z
Maria Hernandez,12278.0,13,944.4615384615385,2004-11-21,High Value,2025-07-01T22:50:16.00517Z
Peter Ferguson,23679.0,25,947.16,2004-11-29,High Value,2025-07-01T22:50:16.00517Z
Sue Taylor,5743.0,7,820.4285714285714,2004-02-10,Regular,2025-07-01T22:50:16.00517Z
Paolo Accorti,15553.0,12,1296.0833333333333,2004-09-09,High Value,2025-07-01T22:50:16.00517Z
Eric Natividad,28614.0,23,1244.0869565217392,2003-10-22,High Value,2025-07-01T22:50:16.00517Z
Janine Labrune,24459.0,26,940.7307692307693,2005-05-31,High Value,2025-07-01T22:50:16.00517Z
Roland Keitel,9424.0,10,942.4,2004-11-05,Regular,2025-07-01T22:50:16.00517Z
Akiko Shimamura,14185.0,18,788.0555555555555,2005-04-22,High Value,2025-07-01T22:50:16.00517Z
Julie Brown,17400.0,16,1087.5,2005-02-23,High Value,2025-07-01T22:50:16.00517Z


In [0]:
# Store processed data in Delta Lake
try:
    # Store customer dimension
    dim_customer.write \
        .format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .save(f"{PROCESSED_DATA_PATH}dim_customer")
    
    # Store product dimension
    dim_product.write \
        .format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .save(f"{PROCESSED_DATA_PATH}dim_product")
    
    # Store fact transactions
    fact_transactions.write \
        .format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .save(f"{PROCESSED_DATA_PATH}fact_transactions")
    
    # Store customer segments
    customer_segments.write \
        .format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .save(f"{PROCESSED_DATA_PATH}customer_segments")
    
    logger.info("All processed data stored in Delta Lake successfully")
    
except Exception as e:
    logger.error(f"Error storing processed data: {str(e)}")
    raise


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:All processed data stored in Delta Lake successfully


In [0]:
#test
df=spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}customer_segments")
df.count()

INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


91

In [0]:
# Load to Redshift
import psycopg2
from pyspark.sql import DataFrameWriter

def load_to_redshift(df, table_name, redshift_config):
    """Load DataFrame to Redshift"""
    try:
        # Convert Spark DataFrame to Pandas for easier Redshift loading
        pandas_df = df.toPandas()
        
        # Connect to Redshift
        conn = psycopg2.connect(
            host=redshift_config["host"],
            port=redshift_config["port"],
            database=redshift_config["database"],
            user=redshift_config["user"],
            password=redshift_config["password"]
        )
        
        cursor = conn.cursor()
        
        # Create table if not exists (simplified schema)
        if table_name == "dim_customer":
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    customer_key VARCHAR(500),
                    first_name VARCHAR(100),
                    last_name VARCHAR(100),
                    full_name VARCHAR(200),
                    email VARCHAR(200),
                    company_name VARCHAR(200),
                    address VARCHAR(500),
                    city VARCHAR(100),
                    state VARCHAR(50),
                    postal_code VARCHAR(20),
                    phone VARCHAR(50),
                    created_date TIMESTAMP,
                    updated_date TIMESTAMP
                );
            """)
        elif table_name == "fact_transactions":
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    order_number BIGINT,
                    customer_name VARCHAR(200),
                    address VARCHAR(500),
                    phone_number VARCHAR(50),
                    product_code VARCHAR(50),
                    quantity INTEGER,
                    unit_price DECIMAL(10,2),
                    total_amount DECIMAL(12,2),
                    deal_size VARCHAR(50),
                    territory VARCHAR(50),
                    order_date DATE,
                    status VARCHAR(50),
                    product_line VARCHAR(100),
                    created_date TIMESTAMP,
                    transaction_type VARCHAR(20),
                    restocking_fee DECIMAL(10,2),
                    net_amount DECIMAL(12,2),
                    is_large_order BOOLEAN
                );
            """)
        elif table_name == "customer_segments":
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    customer_name VARCHAR(200),
                    total_spent DECIMAL(12,2),
                    total_orders INTEGER,
                    avg_order_value DECIMAL(10,2),
                    last_order_date DATE,
                    customer_segment VARCHAR(50),
                    created_date TIMESTAMP
                );
            """)
        
        # Clear existing data
        cursor.execute(f"DELETE FROM {table_name};")
        
        # Insert new data
        for _, row in pandas_df.iterrows():
            placeholders = ', '.join(['%s'] * len(row))
            columns = ', '.join(pandas_df.columns)
            sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            cursor.execute(sql, tuple(row))
        
        conn.commit()
        cursor.close()
        conn.close()
        
        logger.info(f"Successfully loaded {len(pandas_df)} records to Redshift table {table_name}")
        
    except Exception as e:
        logger.error(f"Error loading to Redshift table {table_name}: {str(e)}")
        raise

# Redshift configuration (using secrets)
try:
    redshift_config = {
        "host": "psycho-bunny-workgroup.499897076142.us-east-1.redshift-serverless.amazonaws.com",
        "port": "5439",
        "database": "dev",
        "user": "admin",
        "password": "Tata!8237552399"
    }
    
    # Load key tables to Redshift
    load_to_redshift(dim_customer, "dim_customer", redshift_config)
    load_to_redshift(fact_transactions, "fact_transactions", redshift_config)
    load_to_redshift(customer_segments, "customer_segments", redshift_config)
    
    logger.info("All data successfully loaded to Redshift")
    
except Exception as e:
    logger.warning(f"Redshift loading failed (this is optional): {str(e)}")
    logger.info("Continuing without Redshift - data is still available in Delta Lake")


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clie

In [0]:
# Load additional dimension tables to Redshift (dim_product and dim_calendar)
def load_additional_dimensions_to_redshift(redshift_config):
    """Load dim_product and dim_calendar to Redshift"""
    try:
        # Connect to Redshift
        conn = psycopg2.connect(
        host= "psycho-bunny-workgroup.499897076142.us-east-1.redshift-serverless.amazonaws.com",
        port="5439",
        database= "dev",
        user= "admin",
        password= "Tata!8237552399"
        )
        
        cursor = conn.cursor()
        
        # Create dim_product table schema matching the DataFrame
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS dim_product (
                PRODUCTCODE VARCHAR(50),
                product_key VARCHAR(50),
                product_family VARCHAR(100),
                product_line VARCHAR(100),
                created_date TIMESTAMP
            );
        """)
        
        # Create dim_calendar table schema (using calendar_df columns)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS dim_calendar (
                date_key DATE,
                year INTEGER,
                month INTEGER,
                day INTEGER,
                quarter INTEGER,
                day_of_week INTEGER,
                month_name VARCHAR(20),
                day_name VARCHAR(20),
                created_date TIMESTAMP
            );
        """)
        
        # Clear existing data
        cursor.execute("DELETE FROM dim_product;")
        cursor.execute("DELETE FROM dim_calendar;")
        
        # Load dim_product data
        dim_product_pandas = dim_product.toPandas()
        for _, row in dim_product_pandas.iterrows():
            cursor.execute("""
                INSERT INTO dim_product (PRODUCTCODE, product_key, product_family, product_line, created_date) 
                VALUES (%s, %s, %s, %s, %s)
            """, (row['PRODUCTCODE'], row['product_key'], row['product_family'], row['product_line'], row['created_date']))
        
        # Create simple calendar dimension from calendar_df
        dim_calendar = calendar_df.select(
            col("CALENDAR_DATE").alias("date_key"),
            year(col("CALENDAR_DATE")).alias("year"),
            month(col("CALENDAR_DATE")).alias("month"),
            dayofmonth(col("CALENDAR_DATE")).alias("day"),
            quarter(col("CALENDAR_DATE")).alias("quarter"),
            dayofweek(col("CALENDAR_DATE")).alias("day_of_week"),
            date_format(col("CALENDAR_DATE"), "MMMM").alias("month_name"),
            date_format(col("CALENDAR_DATE"), "EEEE").alias("day_name"),
            current_timestamp().alias("created_date")
        )
        
        # Load dim_calendar data
        dim_calendar_pandas = dim_calendar.toPandas()
        for _, row in dim_calendar_pandas.iterrows():
            cursor.execute("""
                INSERT INTO dim_calendar (date_key, year, month, day, quarter, day_of_week, month_name, day_name, created_date) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (row['date_key'], row['year'], row['month'], row['day'], row['quarter'], row['day_of_week'], row['month_name'], row['day_name'], row['created_date']))
        
        conn.commit()
        cursor.close()
        conn.close()
        
        logger.info(f"Successfully loaded dim_product ({len(dim_product_pandas)} records) and dim_calendar ({len(dim_calendar_pandas)} records) to Redshift")
        
    except Exception as e:
        logger.error(f"Error loading additional dimensions to Redshift: {str(e)}")
        raise

# Load additional dimension tables
try:
    load_additional_dimensions_to_redshift(redshift_config)
    logger.info("All dimension tables loaded to Redshift successfully")
    
except Exception as e:
    logger.warning(f"Additional dimension loading failed (this is optional): {str(e)}")
    logger.info("Main data tables are still available in Redshift and Delta Lake")


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clie

In [0]:
# Data Processing Summary
print("=" * 60)
print("DATA PROCESSING COMPLETED SUCCESSFULLY")
print("=" * 60)

print(f"Customer Dimension: {dim_customer.count()} records")
print(f"Product Dimension: {dim_product.count()} records") 
print(f"Fact Transactions: {fact_transactions.count()} records")
print(f"Customer Segments: {customer_segments.count()} customers")

print("\n🔧 Business Rules Applied:")
print(f"• High-value customer threshold: ${BUSINESS_RULES['high_value_customer_threshold']:,}")
print(f"• Refund restocking fee: {BUSINESS_RULES['refund_restocking_fee_rate']*100}%")
print(f"• Large order threshold: ${BUSINESS_RULES['large_order_threshold']:,}")

print("\nCustomer Segments:")
customer_segments.groupBy("customer_segment").count().show()

print("\nTransaction Types:")
fact_transactions.groupBy("transaction_type").count().show()

print("\nSales by Territory:")
fact_transactions.filter(col("transaction_type") == "SALE") \
    .groupBy("territory") \
    .agg(sum("net_amount").alias("total_revenue")) \
    .orderBy(desc("total_revenue")) \
    .show()

print("Data available in Delta Lake and Redshift")
print(f"Delta Lake location: {PROCESSED_DATA_PATH}")



DATA PROCESSING COMPLETED SUCCESSFULLY


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0


📊 Customer Dimension: 2000 records


INFO:py4j.clientserver:Received command c on object id p0


📦 Product Dimension: 109 records


...

INFO:py4j.clientserver:Received command c on object id p0


+----------------+-----+
|transaction_type|count|
+----------------+-----+
|            SALE| 2823|
+----------------+-----+


🌍 Sales by Territory:


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.java_gateway:Callback Server Shutting Down
INFO:py4j.clientserver:Closing down clientserver connection


+---------+-------------+
|territory|total_revenue|
+---------+-------------+
|   Medium|     138400.0|
|    Small|     128200.0|
|    Large|      15700.0|
+---------+-------------+

✅ Data available in Delta Lake and Redshift (if configured)
📁 Delta Lake location: s3://psycho-bunny-data-lake/processed-data/


INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clientserver:Received command c on object id p0
INFO:py4j.clie

In [0]:
# Load processed data from Delta Lake
try:
    fact_transactions = spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}fact_transactions")
    dim_customer = spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}dim_customer")
    dim_product = spark.read.format("delta").load(f"{PROCESSED_DATA_PATH}dim_product")
    calendar_df = spark.read.format("delta").load(f"{RAW_DATA_PATH}calendar")
    
    logger.info(f"Loaded data: {fact_transactions.count()} transactions, {dim_customer.count()} customers")
    
except Exception as e:
    logger.error(f"❌ Error loading data: {str(e)}")
    raise
